# Internet of Things (2IMN25)
### Machine Learning Assignment

### Introduction

*Add assignment description here*

In [1]:
""" Import all libraries here """
import numpy as np
import os
import math
from datetime import datetime
import pandas as pd
from dateutil import parser


## Reading the dataset

In the code box below, please read in the data-sets into suitable data structures. The key task here would be to contruct the feature vectors which would be fed to the regression algorithm.

The datasets for the weather, rain and energy consumption have been provided in the zip file, extract this zip file into the same folder as the python notebook. 

*Hint : Be sure to check if there are any missing fields in the provided data*

In [2]:
data_path = os.path.join(os.getcwd(),'data')

""" Read in the weather"""
weather_array = pd.read_csv(os.path.join(data_path,'Weather-Eindhoven.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0],
                         usecols = [0,2,4,6,7,8]) #Drop "chill", too many missing values
weather_array  = weather_array.as_matrix()

""" Read in the rain data"""
rain_array = pd.read_csv(os.path.join(data_path,'Rain-Best.csv'),
                         delimiter = ',',
                         header=0,
                         parse_dates = [0])

rain_array = rain_array.as_matrix()

""" Synchronize the 2 data-sets by removing the extra data from the rainfall dataset"""
remove_rows = []
for element,idx in zip(rain_array[:,0],range(len(rain_array[:,0]))):
    if element.minute%10 != 0:
        remove_rows.append(idx)

synced_rain_array = np.delete(rain_array,remove_rows,axis = 0)

print(synced_rain_array.shape)
print(weather_array[:,0].shape)

""" Merge the weather and rain data when time-stamps are equal """
merged_array = []
for rainElem in synced_rain_array:
    try :
        merged_row = []
        valid_row = list(weather_array[:,0]).index(rainElem[0])
        for weatherFeature in weather_array[valid_row]:
            merged_row.append(weatherFeature)
        merged_row.append(rainElem[1])
        merged_array.append(np.asarray(merged_row))
    except ValueError:
        print('Did not find matching timestamp in the weather data, continuing')
        continue

merged_array = np.asarray(merged_array)

(43844, 2)
(43872,)
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matching timestamp in the weather data, continuing
Did not find matchin

In [3]:
""" Clean up the data by removing rows with missing data points """
missing_data_rows = []
for row,idx in zip(merged_array,range(merged_array.shape[0])):
    if row[0].month > 10:
        missing_data_rows.append(idx)
        continue # No energy data for the month of November so delete those too.
    for elem in row[1:]: #Timestamp cannot be "checked"
        if math.isnan(elem):
            missing_data_rows.append(idx)
            break # To prevent the same row from being added multiple times to the list, break when the first "nan" is found
clean_data_array = np.delete(merged_array,missing_data_rows,axis=0)
print(clean_data_array.shape)

(43501, 7)


In [4]:
def create_dataframe(energyDir):
    """ Returns a dataframe that is formed by concatenating all the energy data files """
    fileList = []
    energyDir = os.path.join(os.getcwd(),energyDir)
    dirList = [os.path.join(energyDir, o) for o in os.listdir(energyDir) 
                    if os.path.isdir(os.path.join(energyDir,o))]
    
    sortedDirList = sorted(dirList)
    for direc in sortedDirList:
        dirFiles = [os.path.join(direc, f) for f in os.listdir(direc) 
                    if os.path.isfile(os.path.join(direc,f))]
        for f in dirFiles:
            fileList.append(f)
        
    fileList = sorted(fileList)
    frame = pd.DataFrame()
    frameList = []
    for f in fileList:
        try:
            df = pd.read_csv(f,index_col=None, header=None,parse_dates=[[0,1]],skipfooter=2)
            frameList.append(df)
        except:
            continue
    frame = pd.concat(frameList)
    sparseFrame = frame.drop([2,3,4,5,6,7,8,9,12,13],axis=1)
    sparseFrame.columns = ['TimeStamp','Energy Consumed','Energy Produced']
    return sparseFrame
    
energyFrame = create_dataframe('data/2017')
    


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


In [13]:
energyFrame['TimeStamp'] = pd.to_datetime(energyFrame['TimeStamp'])
energyFrame[(energyFrame['TimeStamp']>clean_data_array[0][0]) & (energyFrame['TimeStamp']<clean_data_array[1][0])]

,TimeStamp,Energy Consumed,Energy Produced
0,2017-01-01 00:00:02,420,0
1,2017-01-01 00:00:12,420,0
2,2017-01-01 00:00:22,420,0
3,2017-01-01 00:00:32,420,0
4,2017-01-01 00:00:42,420,0
5,2017-01-01 00:00:52,420,0
6,2017-01-01 00:01:02,420,0
7,2017-01-01 00:01:12,420,0
8,2017-01-01 00:01:22,530,0
9,2017-01-01 00:01:32,520,0
